<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Xavier", "transactions": [{"transaction-id": 467, "amount": 2694}, {"transaction-id": 511, "amount": 2376}, {"transaction-id": 810, "amount": 2550}, {"transaction-id": 1448, "amount": 2647}, {"transaction-id": 1580, "amount": 2370}, {"transaction-id": 1692, "amount": 2548}, {"transaction-id": 1913, "amount": 2576}, {"transaction-id": 2127, "amount": 2560}, {"transaction-id": 2489, "amount": 2389}, {"transaction-id": 2709, "amount": 2490}, {"transaction-id": 3032, "amount": 2399}, {"transaction-id": 3141, "amount": 2159}, {"transaction-id": 3542, "amount": 2450}, {"transaction-id": 3562, "amount": 2320}, {"transaction-id": 3773, "amount": 2648}, {"transaction-id": 3942, "amount": 2601}, {"transaction-id": 4024, "amount": 2543}, {"transaction-id": 4443, "amount": 2366}, {"transaction-id": 4556, "amount": 2628}, {"transaction-id": 4669, "amount": 2753}, {"transaction-id": 4777, "amount": 2350}, {"transaction-id": 4844, "amount": 2447}, {"transaction-id": 5141, "amount

In [5]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text('s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv',
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [6]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [7]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [8]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Xavier", "transactions": [{"transaction-id": 467, "amount": 2694}, {"transaction-id": 511, "amount": 2376}, {"transaction-id": 810, "amount": 2550}, {"transaction-id": 1448, "amount": 2647}, {"transaction-id": 1580, "amount": 2370}, {"transaction-id": 1692, "amount": 2548}, {"transaction-id": 1913, "amount": 2576}, {"transaction-id": 2127, "amount": 2560}, {"transaction-id": 2489, "amount": 2389}, {"transaction-id": 2709, "amount": 2490}, {"transaction-id": 3032, "amount": 2399}, {"transaction-id": 3141, "amount": 2159}, {"transaction-id": 3542, "amount": 2450}, {"transaction-id": 3562, "amount": 2320}, {"transaction-id": 3773, "amount": 2648}, {"transaction-id": 3942, "amount": 2601}, {"transaction-id": 4024, "amount": 2543}, {"transaction-id": 4443, "amount": 2366}, {"transaction-id": 4556, "amount": 2628}, {"transaction-id": 4669, "amount": 2753}, {"transaction-id": 4777, "amount": 2350}, {"transaction-id": 4844, "amount": 2447}, {"transaction-id": 5141, "amount

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [9]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Xavier',
  'transactions': [{'transaction-id': 467, 'amount': 2694},
   {'transaction-id': 511, 'amount': 2376},
   {'transaction-id': 810, 'amount': 2550},
   {'transaction-id': 1448, 'amount': 2647},
   {'transaction-id': 1580, 'amount': 2370},
   {'transaction-id': 1692, 'amount': 2548},
   {'transaction-id': 1913, 'amount': 2576},
   {'transaction-id': 2127, 'amount': 2560},
   {'transaction-id': 2489, 'amount': 2389},
   {'transaction-id': 2709, 'amount': 2490},
   {'transaction-id': 3032, 'amount': 2399},
   {'transaction-id': 3141, 'amount': 2159},
   {'transaction-id': 3542, 'amount': 2450},
   {'transaction-id': 3562, 'amount': 2320},
   {'transaction-id': 3773, 'amount': 2648},
   {'transaction-id': 3942, 'amount': 2601},
   {'transaction-id': 4024, 'amount': 2543},
   {'transaction-id': 4443, 'amount': 2366},
   {'transaction-id': 4556, 'amount': 2628},
   {'transaction-id': 4669, 'amount': 2753},
   {'transaction-id': 4777, 'amount': 2350},
   {'transa

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [10]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 36,
  'name': 'Alice',
  'transactions': [{'transaction-id': 244, 'amount': 5715},
   {'transaction-id': 921, 'amount': 5349},
   {'transaction-id': 1503, 'amount': 5681},
   {'transaction-id': 2416, 'amount': 5544},
   {'transaction-id': 3293, 'amount': 5608},
   {'transaction-id': 3429, 'amount': 5777},
   {'transaction-id': 4147, 'amount': 5775},
   {'transaction-id': 4926, 'amount': 5714},
   {'transaction-id': 5294, 'amount': 5847},
   {'transaction-id': 5604, 'amount': 5819},
   {'transaction-id': 6205, 'amount': 5859},
   {'transaction-id': 6806, 'amount': 5229},
   {'transaction-id': 6995, 'amount': 5788},
   {'transaction-id': 7113, 'amount': 5332},
   {'transaction-id': 7172, 'amount': 5722},
   {'transaction-id': 7380, 'amount': 5684},
   {'transaction-id': 7479, 'amount': 5392},
   {'transaction-id': 7537, 'amount': 5581},
   {'transaction-id': 7543, 'amount': 5866},
   {'transaction-id': 8992, 'amount': 5833},
   {'transaction-id': 9242, 'amount': 5404},
   {'trans

In [11]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 22},
 {'name': 'Alice', 'count': 13},
 {'name': 'Alice', 'count': 29},
 {'name': 'Alice', 'count': 33},
 {'name': 'Alice', 'count': 78})

In [12]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(22, 13, 29, 33, 78)

In [13]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

36.39142857142857

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [14]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 244, 'amount': 5715},
  {'transaction-id': 921, 'amount': 5349},
  {'transaction-id': 1503, 'amount': 5681},
  {'transaction-id': 2416, 'amount': 5544},
  {'transaction-id': 3293, 'amount': 5608},
  {'transaction-id': 3429, 'amount': 5777},
  {'transaction-id': 4147, 'amount': 5775},
  {'transaction-id': 4926, 'amount': 5714},
  {'transaction-id': 5294, 'amount': 5847},
  {'transaction-id': 5604, 'amount': 5819},
  {'transaction-id': 6205, 'amount': 5859},
  {'transaction-id': 6806, 'amount': 5229},
  {'transaction-id': 6995, 'amount': 5788},
  {'transaction-id': 7113, 'amount': 5332},
  {'transaction-id': 7172, 'amount': 5722},
  {'transaction-id': 7380, 'amount': 5684},
  {'transaction-id': 7479, 'amount': 5392},
  {'transaction-id': 7537, 'amount': 5581},
  {'transaction-id': 7543, 'amount': 5866},
  {'transaction-id': 8992, 'amount': 5833},
  {'transaction-id': 9242, 'amount': 5404},
  {'transaction-id': 9435, 'amount': 5731}],
 [{'transaction-id': 68, 'amount'

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 244, 'amount': 5715},
 {'transaction-id': 921, 'amount': 5349},
 {'transaction-id': 1503, 'amount': 5681})

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(5715, 5349, 5681)

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

1380.9596451283662

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [18]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [19]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [20]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [21]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [22]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 84), ('Alice', 84), ('Alice', 84), ('Alice', 98), ('Bob', 72), ('Bob', 72), ('Bob', 72), ('Bob', 86), ('Charlie', 89), ('Charlie', 90), ('Charlie', 92), ('Charlie', 107), ('Dan', 156), ('Dan', 156), ('Dan', 156), ('Dan', 181), ('Edith', 159), ('Edith', 160), ('Edith', 163), ('Edith', 184), ('Frank', 67), ('Frank', 70), ('Frank', 72), ('Frank', 80), ('George', 78), ('George', 79), ('George', 82), ('George', 94), ('Hannah', 144), ('Hannah', 144), ('Hannah', 144), ('Hannah', 168), ('Ingrid', 104), ('Ingrid', 104), ('Ingrid', 107), ('Ingrid', 124), ('Jerry', 71), ('Jerry', 72), ('Jerry', 72), ('Jerry', 84), ('Kevin', 119), ('Kevin', 119), ('Kevin', 119), ('Kevin', 138), ('Laura', 115), ('Laura', 117), ('Laura', 118), ('Laura', 133), ('Michael', 96), ('Michael', 96), ('Michael', 208), ('Norbert', 118), ('Norbert', 135), ('Norbert', 232), ('Oliver', 107), ('Oliver', 108), ('Oliver', 108), ('Oliver', 126), ('Patricia', 52), ('Patricia', 53), ('Patricia', 55), ('Patricia', 66), ('Qu

In [23]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 350), ('Bob', 302), ('Charlie', 378), ('Dan', 649), ('Edith', 666), ('Frank', 289), ('George', 333), ('Hannah', 600), ('Ingrid', 439), ('Jerry', 299), ('Kevin', 495), ('Laura', 483), ('Michael', 400), ('Norbert', 485), ('Oliver', 449), ('Patricia', 226), ('Quinn', 530), ('Ray', 475), ('Sarah', 500), ('Tim', 600), ('Ursula', 300), ('Victor', 246), ('Wendy', 858), ('Xavier', 499), ('Yvonne', 745), ('Zelda', 476)]
CPU times: user 247 ms, sys: 9.39 ms, total: 257 ms
Wall time: 790 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [24]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [25]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Xavier,"[{'transaction-id': 467, 'amount': 2694}, {'tr..."
1,1,Xavier,"[{'transaction-id': 212, 'amount': 3673}, {'tr..."
2,2,Ursula,"[{'transaction-id': 5, 'amount': -299}, {'tran..."
3,3,Wendy,"[{'transaction-id': 457, 'amount': 5402}, {'tr..."
4,4,Tim,"[{'transaction-id': 10, 'amount': 203}, {'tran..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [26]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 354 ms, sys: 40.3 ms, total: 394 ms
Wall time: 2.31 s


name
Alice      350
Bob        302
Charlie    378
Dan        649
Edith      666
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [27]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Xavier', 'amount': 2694, 'transaction-id': 467},
 {'id': 0, 'name': 'Xavier', 'amount': 2376, 'transaction-id': 511},
 {'id': 0, 'name': 'Xavier', 'amount': 2550, 'transaction-id': 810})

In [28]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Xavier,2694,467
1,0,Xavier,2376,511
2,0,Xavier,2550,810
3,0,Xavier,2647,1448
4,0,Xavier,2370,1580


In [29]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 393 ms, sys: 12.5 ms, total: 405 ms
Wall time: 1.97 s


name
Alice       12737
Bob         17780
Charlie     11831
Dan         25002
Edith       29216
Frank        7105
George      15865
Hannah      29255
Ingrid      21436
Jerry        6540
Kevin       20947
Laura       14147
Michael     28429
Norbert     22895
Oliver      18862
Patricia     8546
Quinn       13105
Ray         14894
Sarah       20413
Tim         20592
Ursula      22915
Victor       8884
Wendy       36378
Xavier      20344
Yvonne      31315
Zelda       20567
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [30]:
client.shutdown()